In [1]:
import os
import pandas as pd
import numpy as np

from PIL import Image

data = pd.read_csv('all_.csv', index_col=0)

In [2]:
from ipywidgets import Textarea

def get_input(change):
    global Input
    Input = change["new"]

textarea = Textarea()
textarea.observe(get_input, names='value')
display(textarea)

Textarea(value='')

In [3]:
IN = iter(Input.split('\n')).__next__
def input():
    return IN()

N, M = map(int, input().split())

Graph = [[] for _ in range(4800)]

for i in range(M):
    a, b, direction = map(int, input().split())
    Graph[a].append((b, direction))

In [4]:
# 画像データがあるディレクトリを指定(localで動かす場合)
os.chdir("D:/worldmodeldata")

In [5]:
def node_int_to_string(node):
    node = str(node)
    temp = len(node)
    fill = 6 - temp
    fill_text = "0" * fill
    node = fill_text + node
    return node

def return_image_and_moveable_node(image, action):
    """
    Graph
    input image(node), action
    return image, next_node, and moveable_direction

    """
    current_node = int(image[:6])
    moveable_nodes = Graph[current_node]
    next_node = float('inf')
    
    
    print(f"今のnodeは{current_node}で動ける箇所は{Graph[current_node]}")
    
    for i in moveable_nodes:
        if i[1] == action:
            next_node = i[0]
    
    
    
    next_image = node_int_to_string(next_node) + "_" + str(action) + ".jpg"
    _next_image = Image.open(next_image)
    
    # ここでappendしてます。外に書いたほうがいいかもしれません。
    pictures.append(_next_image)
    next_moveable_nodes = Graph[next_node]
    
    return next_image, next_node, next_moveable_nodes

In [7]:
pictures = []
# next_image, next_node, next_moveable_nodes = return_image_and_moveable_node("001257_0.jpg", 1)

next_image = "001035_0.jpg"

for i in range(50):
    next_image, next_node, next_moveable_nodes = return_image_and_moveable_node(next_image, 1)

今のnodeは1035で動ける箇所は[(1228, 1), (1036, 0), (1034, 4)]
今のnodeは1228で動ける箇所は[(1229, 1), (1035, 3)]
今のnodeは1229で動ける箇所は[(1230, 1), (1228, 3)]
今のnodeは1230で動ける箇所は[(1231, 1), (1229, 3)]
今のnodeは1231で動ける箇所は[(1232, 1), (1230, 3)]
今のnodeは1232で動ける箇所は[(1233, 1), (1231, 3)]
今のnodeは1233で動ける箇所は[(1234, 1), (1232, 3)]
今のnodeは1234で動ける箇所は[(1235, 1), (1233, 3)]
今のnodeは1235で動ける箇所は[(1236, 1), (1234, 3)]
今のnodeは1236で動ける箇所は[(1237, 1), (1235, 3)]
今のnodeは1237で動ける箇所は[(1238, 1), (1236, 3)]
今のnodeは1238で動ける箇所は[(1239, 1), (1237, 3)]
今のnodeは1239で動ける箇所は[(609, 1), (1238, 3)]
今のnodeは609で動ける箇所は[(1240, 1), (1239, 3), (610, 0), (608, 4)]
今のnodeは1240で動ける箇所は[(1241, 1), (609, 3)]
今のnodeは1241で動ける箇所は[(1242, 1), (1240, 3)]
今のnodeは1242で動ける箇所は[(1243, 1), (1241, 3)]
今のnodeは1243で動ける箇所は[(1244, 1), (1242, 3)]
今のnodeは1244で動ける箇所は[(1245, 1), (1243, 3)]
今のnodeは1245で動ける箇所は[(1246, 1), (1244, 3)]
今のnodeは1246で動ける箇所は[(1247, 1), (1245, 3)]
今のnodeは1247で動ける箇所は[(1248, 1), (1246, 3)]
今のnodeは1248で動ける箇所は[(1249, 1), (1247, 3)]
今のnodeは1249で動ける箇所は[(1250, 1),

In [8]:
pictures[0].save('anime.gif',save_all=True, append_images=pictures[1:], optimize=False, duration=500, loop=0)